# Character-Level LSTM in PyTorch

In this notebook, I'll construct a character-level LSTM with PyTorch. The network will train character by character on some text, then generate new text character by character. As an example, I will train on Anna Karenina. **This model will be able to generate new text based on the text from the book!**

This network is based off of Andrej Karpathy's [post on RNNs](http://karpathy.github.io/2015/05/21/rnn-effectiveness/) and [implementation in Torch](https://github.com/karpathy/char-rnn). Below is the general architecture of the character-wise RNN.

<img src="assets/charseq.jpeg" width="500">

First let's load in our required resources for data loading and model creation.

In [1]:
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F

## Load in Data

Then, we'll load the Anna Karenina text file and convert it into integers for our network to use. 

In [2]:
# open text file and read in data as `text`
with open('data/anna.txt', 'r') as f:
    text = f.read()

Let's check out the first 100 characters, make sure everything is peachy. According to the [American Book Review](http://americanbookreview.org/100bestlines.asp), this is the 6th best first line of a book ever.

In [3]:
text[:100]

'Chapter 1\n\n\nHappy families are all alike; every unhappy family is unhappy in its own\nway.\n\nEverythin'

### Tokenization

In the cells, below, I'm creating a couple of **dictionaries** to convert the characters to and from integers. Encoding the characters as integers makes it easier to use as input in the network.

In [4]:
# encode the text and map each character to an integer and vice versa

# we create two dictionaries:
# 1. int2char, which maps integers to characters
# 2. char2int, which maps characters to unique integers
chars = tuple(set(text))
int2char = dict(enumerate(chars))
char2int = {ch: ii for ii, ch in int2char.items()}

# encode the text
encoded = np.array([char2int[ch] for ch in text])

And we can see those same characters from above, encoded as integers.

In [5]:
encoded[:100]

array([70, 13, 30, 16, 74, 82, 45, 17, 47,  7,  7,  7, 60, 30, 16, 16, 21,
       17, 68, 30,  0, 57, 77, 57, 82, 76, 17, 30, 45, 82, 17, 30, 77, 77,
       17, 30, 77, 57, 25, 82, 34, 17, 82, 51, 82, 45, 21, 17, 37, 14, 13,
       30, 16, 16, 21, 17, 68, 30,  0, 57, 77, 21, 17, 57, 76, 17, 37, 14,
       13, 30, 16, 16, 21, 17, 57, 14, 17, 57, 74, 76, 17,  5, 78, 14,  7,
       78, 30, 21, 41,  7,  7,  2, 51, 82, 45, 21, 74, 13, 57, 14])

## Pre-processing the data

As you can see in our char-RNN image above, our LSTM expects an input that is **one-hot encoded** meaning that each character is converted into an integer (via our created dictionary) and *then* converted into a column vector where only it's corresponding integer index will have the value of 1 and the rest of the vector will be filled with 0's. Since we're one-hot encoding the data, let's make a function to do that!


In [6]:
def one_hot_encode(arr, n_labels):
    
    # Initialize the encoded array
    one_hot = np.zeros((arr.size, n_labels), dtype=np.float32)
    
    # Fill the appropriate elements with ones
    one_hot[np.arange(one_hot.shape[0]), arr.flatten()] = 1.
    
    # Finally reshape it to get back to the original array
    one_hot = one_hot.reshape((*arr.shape, n_labels))
    
    return one_hot

In [7]:
# check that the function works as expected
test_seq = np.array([[3, 5, 1]])
one_hot = one_hot_encode(test_seq, 8)

print(one_hot)

[[[0. 0. 0. 1. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 1. 0. 0.]
  [0. 1. 0. 0. 0. 0. 0. 0.]]]


## Making training mini-batches


To train on this data, we also want to create mini-batches for training. Remember that we want our batches to be multiple sequences of some desired number of sequence steps. Considering a simple example, our batches would look like this:

<img src="assets/sequence_batching@1x.png" width=500px>


<br>

In this example, we'll take the encoded characters (passed in as the `arr` parameter) and split them into multiple sequences, given by `batch_size`. Each of our sequences will be `seq_length` long.

### Creating Batches

**1. The first thing we need to do is discard some of the text so we only have completely full mini-batches. **

Each batch contains $N \times M$ characters, where $N$ is the batch size (the number of sequences in a batch) and $M$ is the seq_length or number of time steps in a sequence. Then, to get the total number of batches, $K$, that we can make from the array `arr`, you divide the length of `arr` by the number of characters per batch. Once you know the number of batches, you can get the total number of characters to keep from `arr`, $N * M * K$.

**2. After that, we need to split `arr` into $N$ batches. ** 

You can do this using `arr.reshape(size)` where `size` is a tuple containing the dimensions sizes of the reshaped array. We know we want $N$ sequences in a batch, so let's make that the size of the first dimension. For the second dimension, you can use `-1` as a placeholder in the size, it'll fill up the array with the appropriate data for you. After this, you should have an array that is $N \times (M * K)$.

**3. Now that we have this array, we can iterate through it to get our mini-batches. **

The idea is each batch is a $N \times M$ window on the $N \times (M * K)$ array. For each subsequent batch, the window moves over by `seq_length`. We also want to create both the input and target arrays. Remember that the targets are just the inputs shifted over by one character. The way I like to do this window is use `range` to take steps of size `n_steps` from $0$ to `arr.shape[1]`, the total number of tokens in each sequence. That way, the integers you get from `range` always point to the start of a batch, and each window is `seq_length` wide.

> **TODO:** Write the code for creating batches in the function below. The exercises in this notebook _will not be easy_. I've provided a notebook with solutions alongside this notebook. If you get stuck, checkout the solutions. The most important thing is that you don't copy and paste the code into here, **type out the solution code yourself.**

In [8]:
def get_batches(arr, batch_size, seq_length):
    '''Create a generator that returns batches of size
       batch_size x seq_length from arr.
       
       Arguments
       ---------
       arr: Array you want to make batches from
       batch_size: Batch size, the number of sequences per batch
       seq_length: Number of encoded chars in a sequence
    '''
    
    ## DONE: Get the number of batches we can make
    n_batches = len(arr)//(batch_size * seq_length)
    
    ## DONE: Keep only enough characters to make full batches
    arr = arr[:n_batches * batch_size * seq_length]
    
    ## DONE: Reshape into batch_size rows
    arr = arr.reshape((batch_size, -1))
    
    ## TODO: Iterate over the batches using a window of size seq_length
    for n in range(0, arr.shape[1], seq_length):
        # The features
        x = arr[:, n:n+seq_length]
        # The targets, shifted by one
        y = np.zeros_like(x)
        try:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, n+seq_length]
        except IndexError:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, 0]
        yield x, y

### Test Your Implementation

Now I'll make some data sets and we can check out what's going on as we batch data. Here, as an example, I'm going to use a batch size of 8 and 50 sequence steps.

In [9]:
batches = get_batches(encoded, 8, 50)
x, y = next(batches)

In [10]:
# printing out the first 10 items in a sequence
print('x\n', x[:10, :10])
print('\ny\n', y[:10, :10])

x
 [[70 13 30 16 74 82 45 17 47  7]
 [76  5 14 17 74 13 30 74 17 30]
 [82 14 35 17  5 45 17 30 17 68]
 [76 17 74 13 82 17 56 13 57 82]
 [17 76 30 78 17 13 82 45 17 74]
 [56 37 76 76 57  5 14 17 30 14]
 [17 52 14 14 30 17 13 30 35 17]
 [53 62 77  5 14 76 25 21 41 17]]

y
 [[13 30 16 74 82 45 17 47  7  7]
 [ 5 14 17 74 13 30 74 17 30 74]
 [14 35 17  5 45 17 30 17 68  5]
 [17 74 13 82 17 56 13 57 82 68]
 [76 30 78 17 13 82 45 17 74 82]
 [37 76 76 57  5 14 17 30 14 35]
 [52 14 14 30 17 13 30 35 17 76]
 [62 77  5 14 76 25 21 41 17  8]]


If you implemented `get_batches` correctly, the above output should look something like 
```
x
 [[25  8 60 11 45 27 28 73  1  2]
 [17  7 20 73 45  8 60 45 73 60]
 [27 20 80 73  7 28 73 60 73 65]
 [17 73 45  8 27 73 66  8 46 27]
 [73 17 60 12 73  8 27 28 73 45]
 [66 64 17 17 46  7 20 73 60 20]
 [73 76 20 20 60 73  8 60 80 73]
 [47 35 43  7 20 17 24 50 37 73]]

y
 [[ 8 60 11 45 27 28 73  1  2  2]
 [ 7 20 73 45  8 60 45 73 60 45]
 [20 80 73  7 28 73 60 73 65  7]
 [73 45  8 27 73 66  8 46 27 65]
 [17 60 12 73  8 27 28 73 45 27]
 [64 17 17 46  7 20 73 60 20 80]
 [76 20 20 60 73  8 60 80 73 17]
 [35 43  7 20 17 24 50 37 73 36]]
 ```
 although the exact numbers may be different. Check to make sure the data is shifted over one step for `y`.

---
## Defining the network with PyTorch

Below is where you'll define the network.

<img src="assets/charRNN.png" width=500px>

Next, you'll use PyTorch to define the architecture of the network. We start by defining the layers and operations we want. Then, define a method for the forward pass. You've also been given a method for predicting characters.

### Model Structure

In `__init__` the suggested structure is as follows:
* Create and store the necessary dictionaries (this has been done for you)
* Define an LSTM layer that takes as params: an input size (the number of characters), a hidden layer size `n_hidden`, a number of layers `n_layers`, a dropout probability `drop_prob`, and a batch_first boolean (True, since we are batching)
* Define a dropout layer with `drop_prob`
* Define a fully-connected layer with params: input size `n_hidden` and output size (the number of characters)
* Finally, initialize the weights (again, this has been given)

Note that some parameters have been named and given in the `__init__` function, and we use them and store them by doing something like `self.drop_prob = drop_prob`.

---
### LSTM Inputs/Outputs

You can create a basic [LSTM layer](https://pytorch.org/docs/stable/nn.html#lstm) as follows

```python
self.lstm = nn.LSTM(input_size, n_hidden, n_layers, 
                            dropout=drop_prob, batch_first=True)
```

where `input_size` is the number of characters this cell expects to see as sequential input, and `n_hidden` is the number of units in the hidden layers in the cell. And we can add dropout by adding a dropout parameter with a specified probability; this will automatically add dropout to the inputs or outputs. Finally, in the `forward` function, we can stack up the LSTM cells into layers using `.view`. With this, you pass in a list of cells and it will send the output of one cell into the next cell.

We also need to create an initial hidden state of all zeros. This is done like so

```python
self.init_hidden()
```

In [11]:
# check if GPU is available
train_on_gpu = torch.cuda.is_available()
if(train_on_gpu):
    print('Training on GPU!')
else: 
    print('No GPU available, training on CPU; consider making n_epochs very small.')

Training on GPU!


In [18]:
class CharRNN(nn.Module):
    
    def __init__(self, tokens, n_hidden=256, n_layers=2,
                               drop_prob=0.5, lr=0.001):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers  = n_layers
        self.n_hidden  = n_hidden
        self.lr        = lr
        
        # creating character dictionaries
        self.chars    = tokens
        self.int2char = dict(enumerate(self.chars))
        self.char2int = {ch: ii for ii, ch in self.int2char.items()}
        
        ## DONE: define the layers of the model
        self.lstm = nn.LSTM(len(self.chars), n_hidden, n_layers, 
                            dropout=drop_prob, batch_first=True)
      
        self.dropout = nn.Dropout(drop_prob)
        
        self.fc      = nn.Linear(n_hidden, len(self.chars))
    
    
    def forward(self, x, hidden):
        ''' Forward pass through the network. 
            These inputs are x, and the hidden/cell state `hidden`. '''
                
        ## DONE: Get the outputs and the new hidden state from the lstm
        r_output, hidden = self.lstm(x, hidden)
        
        ## DONE: pass through a dropout layer
        out = self.dropout(r_output)
        
        # Stack up LSTM outputs using view
        # you may need to use contiguous to reshape the output
        out = out.contiguous().view(-1, self.n_hidden)
        
        ## DONE: put x through the fully-connected layer
        out = self.fc(out)
        
        # return the final output and the hidden state
        return out, hidden
    
    
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda(),
                      weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),
                      weight.new(self.n_layers, batch_size, self.n_hidden).zero_())
        
        return hidden
        

## Time to train

The train function gives us the ability to set the number of epochs, the learning rate, and other parameters.

Below we're using an Adam optimizer and cross entropy loss since we are looking at character class scores as output. We calculate the loss and perform backpropagation, as usual!

A couple of details about training: 
>* Within the batch loop, we detach the hidden state from its history; this time setting it equal to a new *tuple* variable because an LSTM has a hidden state that is a tuple of the hidden and cell states.
* We use [`clip_grad_norm_`](https://pytorch.org/docs/stable/_modules/torch/nn/utils/clip_grad.html) to help prevent exploding gradients.

In [19]:
def train(net, data, epochs=10, batch_size=10, seq_length=50, lr=0.001, clip=5, val_frac=0.1, print_every=10):
    ''' Training a network 
    
        Arguments
        ---------
        
        net: CharRNN network
        data: text data to train the network
        epochs: Number of epochs to train
        batch_size: Number of mini-sequences per mini-batch, aka batch size
        seq_length: Number of character steps per mini-batch
        lr: learning rate
        clip: gradient clipping
        val_frac: Fraction of data to hold out for validation
        print_every: Number of steps for printing training and validation loss
    
    '''
    net.train()
    
    opt = torch.optim.Adam(net.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    
    # create training and validation data
    val_idx = int(len(data)*(1-val_frac))
    data, val_data = data[:val_idx], data[val_idx:]
    
    if(train_on_gpu):
        net.cuda()
    
    counter = 0
    n_chars = len(net.chars)
    for e in range(epochs):
        # initialize hidden state
        h = net.init_hidden(batch_size)
        
        for x, y in get_batches(data, batch_size, seq_length):
            counter += 1
            
            # One-hot encode our data and make them Torch tensors
            x = one_hot_encode(x, n_chars)
            inputs, targets = torch.from_numpy(x), torch.from_numpy(y)
            
            if(train_on_gpu):
                inputs, targets = inputs.cuda(), targets.cuda()

            # Creating new variables for the hidden state, otherwise
            # we'd backprop through the entire training history
            h = tuple([each.data for each in h])

            # zero accumulated gradients
            net.zero_grad()
            
            # get the output from the model
            output, h = net(inputs, h)
            
            # calculate the loss and perform backprop
            loss = criterion(output, targets.view(batch_size*seq_length).long())
            loss.backward()
            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
            nn.utils.clip_grad_norm_(net.parameters(), clip)
            opt.step()
            
            # loss stats
            if counter % print_every == 0:
                # Get validation loss
                val_h = net.init_hidden(batch_size)
                val_losses = []
                net.eval()
                for x, y in get_batches(val_data, batch_size, seq_length):
                    # One-hot encode our data and make them Torch tensors
                    x = one_hot_encode(x, n_chars)
                    x, y = torch.from_numpy(x), torch.from_numpy(y)
                    
                    # Creating new variables for the hidden state, otherwise
                    # we'd backprop through the entire training history
                    val_h = tuple([each.data for each in val_h])
                    
                    inputs, targets = x, y
                    if(train_on_gpu):
                        inputs, targets = inputs.cuda(), targets.cuda()

                    output, val_h = net(inputs, val_h)
                    val_loss = criterion(output, targets.view(batch_size*seq_length).long())
                
                    val_losses.append(val_loss.item())
                
                net.train() # reset to train mode after iterationg through validation data
                
                print("Epoch: {}/{}...".format(e+1, epochs),
                      "Step: {}...".format(counter),
                      "Loss: {:.4f}...".format(loss.item()),
                      "Val Loss: {:.4f}".format(np.mean(val_losses)))

## Instantiating the model

Now we can actually train the network. First we'll create the network itself, with some given hyperparameters. Then, define the mini-batches sizes, and start training!

In [20]:
## DONE: set your model hyperparameters
# define and print the net
n_hidden=256
n_layers=2

net = CharRNN(chars, n_hidden, n_layers)
print(net)

CharRNN(
  (lstm): LSTM(83, 256, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=256, out_features=83, bias=True)
)


### Set your training hyperparameters!

In [21]:
batch_size = 10
seq_length = 50
n_epochs   =  5 # start small if you are just testing initial behavior

# train the model
train(net, encoded, epochs=n_epochs, batch_size=batch_size, seq_length=seq_length, lr=0.001, print_every=10)

Epoch: 1/5... Step: 10... Loss: 3.3543... Val Loss: 3.2283
Epoch: 1/5... Step: 20... Loss: 3.2244... Val Loss: 3.1495
Epoch: 1/5... Step: 30... Loss: 3.1547... Val Loss: 3.1362
Epoch: 1/5... Step: 40... Loss: 3.1285... Val Loss: 3.1343
Epoch: 1/5... Step: 50... Loss: 3.1371... Val Loss: 3.1309
Epoch: 1/5... Step: 60... Loss: 3.1204... Val Loss: 3.1309
Epoch: 1/5... Step: 70... Loss: 3.0806... Val Loss: 3.1306
Epoch: 1/5... Step: 80... Loss: 3.1097... Val Loss: 3.1318
Epoch: 1/5... Step: 90... Loss: 3.1787... Val Loss: 3.1314
Epoch: 1/5... Step: 100... Loss: 3.0731... Val Loss: 3.1333
Epoch: 1/5... Step: 110... Loss: 3.0405... Val Loss: 3.1284
Epoch: 1/5... Step: 120... Loss: 3.1164... Val Loss: 3.1287
Epoch: 1/5... Step: 130... Loss: 2.9903... Val Loss: 3.1283
Epoch: 1/5... Step: 140... Loss: 3.1274... Val Loss: 3.1266
Epoch: 1/5... Step: 150... Loss: 3.1495... Val Loss: 3.1220
Epoch: 1/5... Step: 160... Loss: 3.0770... Val Loss: 3.1205
Epoch: 1/5... Step: 170... Loss: 2.9989... Val Lo

Epoch: 1/5... Step: 1380... Loss: 2.1058... Val Loss: 2.0948
Epoch: 1/5... Step: 1390... Loss: 2.1571... Val Loss: 2.1000
Epoch: 1/5... Step: 1400... Loss: 2.0033... Val Loss: 2.0959
Epoch: 1/5... Step: 1410... Loss: 2.1057... Val Loss: 2.0879
Epoch: 1/5... Step: 1420... Loss: 2.1114... Val Loss: 2.0897
Epoch: 1/5... Step: 1430... Loss: 2.1348... Val Loss: 2.0823
Epoch: 1/5... Step: 1440... Loss: 2.1213... Val Loss: 2.0813
Epoch: 1/5... Step: 1450... Loss: 2.2184... Val Loss: 2.0795
Epoch: 1/5... Step: 1460... Loss: 2.0732... Val Loss: 2.0917
Epoch: 1/5... Step: 1470... Loss: 2.1597... Val Loss: 2.0764
Epoch: 1/5... Step: 1480... Loss: 2.1303... Val Loss: 2.0738
Epoch: 1/5... Step: 1490... Loss: 2.1749... Val Loss: 2.0734
Epoch: 1/5... Step: 1500... Loss: 2.2195... Val Loss: 2.0714
Epoch: 1/5... Step: 1510... Loss: 2.0490... Val Loss: 2.0707
Epoch: 1/5... Step: 1520... Loss: 2.1343... Val Loss: 2.0617
Epoch: 1/5... Step: 1530... Loss: 2.1017... Val Loss: 2.0615
Epoch: 1/5... Step: 1540

Epoch: 1/5... Step: 2730... Loss: 1.7259... Val Loss: 1.8135
Epoch: 1/5... Step: 2740... Loss: 1.8025... Val Loss: 1.8125
Epoch: 1/5... Step: 2750... Loss: 1.7856... Val Loss: 1.8177
Epoch: 1/5... Step: 2760... Loss: 1.8486... Val Loss: 1.8088
Epoch: 1/5... Step: 2770... Loss: 1.8500... Val Loss: 1.8089
Epoch: 1/5... Step: 2780... Loss: 1.6309... Val Loss: 1.8053
Epoch: 1/5... Step: 2790... Loss: 1.9041... Val Loss: 1.8047
Epoch: 1/5... Step: 2800... Loss: 1.7882... Val Loss: 1.8062
Epoch: 1/5... Step: 2810... Loss: 1.7959... Val Loss: 1.7981
Epoch: 1/5... Step: 2820... Loss: 1.6815... Val Loss: 1.7996
Epoch: 1/5... Step: 2830... Loss: 1.8050... Val Loss: 1.7987
Epoch: 1/5... Step: 2840... Loss: 2.0134... Val Loss: 1.7924
Epoch: 1/5... Step: 2850... Loss: 2.0627... Val Loss: 1.7934
Epoch: 1/5... Step: 2860... Loss: 1.6211... Val Loss: 1.7944
Epoch: 1/5... Step: 2870... Loss: 1.8277... Val Loss: 1.7902
Epoch: 1/5... Step: 2880... Loss: 1.8947... Val Loss: 1.7865
Epoch: 1/5... Step: 2890

Epoch: 2/5... Step: 4080... Loss: 1.5427... Val Loss: 1.6702
Epoch: 2/5... Step: 4090... Loss: 1.6089... Val Loss: 1.6710
Epoch: 2/5... Step: 4100... Loss: 1.6542... Val Loss: 1.6710
Epoch: 2/5... Step: 4110... Loss: 1.6401... Val Loss: 1.6651
Epoch: 2/5... Step: 4120... Loss: 1.6289... Val Loss: 1.6680
Epoch: 2/5... Step: 4130... Loss: 1.5950... Val Loss: 1.6671
Epoch: 2/5... Step: 4140... Loss: 1.7688... Val Loss: 1.6653
Epoch: 2/5... Step: 4150... Loss: 1.4846... Val Loss: 1.6673
Epoch: 2/5... Step: 4160... Loss: 1.5898... Val Loss: 1.6698
Epoch: 2/5... Step: 4170... Loss: 1.7946... Val Loss: 1.6626
Epoch: 2/5... Step: 4180... Loss: 1.7844... Val Loss: 1.6682
Epoch: 2/5... Step: 4190... Loss: 1.7077... Val Loss: 1.6607
Epoch: 2/5... Step: 4200... Loss: 1.6801... Val Loss: 1.6589
Epoch: 2/5... Step: 4210... Loss: 1.5468... Val Loss: 1.6632
Epoch: 2/5... Step: 4220... Loss: 1.5964... Val Loss: 1.6558
Epoch: 2/5... Step: 4230... Loss: 1.6781... Val Loss: 1.6612
Epoch: 2/5... Step: 4240

Epoch: 2/5... Step: 5430... Loss: 1.7024... Val Loss: 1.6002
Epoch: 2/5... Step: 5440... Loss: 1.5845... Val Loss: 1.5964
Epoch: 2/5... Step: 5450... Loss: 1.5856... Val Loss: 1.5944
Epoch: 2/5... Step: 5460... Loss: 1.6080... Val Loss: 1.5971
Epoch: 2/5... Step: 5470... Loss: 1.5513... Val Loss: 1.5936
Epoch: 2/5... Step: 5480... Loss: 1.5419... Val Loss: 1.5937
Epoch: 2/5... Step: 5490... Loss: 1.5299... Val Loss: 1.5951
Epoch: 2/5... Step: 5500... Loss: 1.6759... Val Loss: 1.5931
Epoch: 2/5... Step: 5510... Loss: 1.8899... Val Loss: 1.5874
Epoch: 2/5... Step: 5520... Loss: 1.5495... Val Loss: 1.5909
Epoch: 2/5... Step: 5530... Loss: 1.5991... Val Loss: 1.5899
Epoch: 2/5... Step: 5540... Loss: 1.6999... Val Loss: 1.5864
Epoch: 2/5... Step: 5550... Loss: 1.6449... Val Loss: 1.5858
Epoch: 2/5... Step: 5560... Loss: 1.4421... Val Loss: 1.5885
Epoch: 2/5... Step: 5570... Loss: 1.5176... Val Loss: 1.5924
Epoch: 2/5... Step: 5580... Loss: 1.6510... Val Loss: 1.5880
Epoch: 2/5... Step: 5590

...

Epoch: 4/5... Step: 12140... Loss: 1.3302... Val Loss: 1.4514
Epoch: 4/5... Step: 12150... Loss: 1.3333... Val Loss: 1.4543
Epoch: 4/5... Step: 12160... Loss: 1.5238... Val Loss: 1.4573
Epoch: 4/5... Step: 12170... Loss: 1.6685... Val Loss: 1.4557
Epoch: 4/5... Step: 12180... Loss: 1.5064... Val Loss: 1.4543
Epoch: 4/5... Step: 12190... Loss: 1.6108... Val Loss: 1.4589
Epoch: 4/5... Step: 12200... Loss: 1.4759... Val Loss: 1.4546
Epoch: 4/5... Step: 12210... Loss: 1.4254... Val Loss: 1.4551
Epoch: 4/5... Step: 12220... Loss: 1.3686... Val Loss: 1.4590
Epoch: 4/5... Step: 12230... Loss: 1.5156... Val Loss: 1.4556
Epoch: 4/5... Step: 12240... Loss: 1.3954... Val Loss: 1.4595
Epoch: 4/5... Step: 12250... Loss: 1.6408... Val Loss: 1.4618
Epoch: 4/5... Step: 12260... Loss: 1.4207... Val Loss: 1.4602
Epoch: 4/5... Step: 12270... Loss: 1.4505... Val Loss: 1.4581
Epoch: 4/5... Step: 12280... Loss: 1.3831... Val Loss: 1.4565
Epoch: 4/5... Step: 12290... Loss: 1.4975... Val Loss: 1.4566
Epoch: 4

Epoch: 4/5... Step: 13470... Loss: 1.4569... Val Loss: 1.4525
Epoch: 4/5... Step: 13480... Loss: 1.4983... Val Loss: 1.4494
Epoch: 4/5... Step: 13490... Loss: 1.4476... Val Loss: 1.4442
Epoch: 4/5... Step: 13500... Loss: 1.4769... Val Loss: 1.4440
Epoch: 4/5... Step: 13510... Loss: 1.4784... Val Loss: 1.4428
Epoch: 4/5... Step: 13520... Loss: 1.4615... Val Loss: 1.4452
Epoch: 4/5... Step: 13530... Loss: 1.4072... Val Loss: 1.4457
Epoch: 4/5... Step: 13540... Loss: 1.4839... Val Loss: 1.4501
Epoch: 4/5... Step: 13550... Loss: 1.5040... Val Loss: 1.4479
Epoch: 4/5... Step: 13560... Loss: 1.5707... Val Loss: 1.4461
Epoch: 4/5... Step: 13570... Loss: 1.3180... Val Loss: 1.4444
Epoch: 4/5... Step: 13580... Loss: 1.4609... Val Loss: 1.4414
Epoch: 4/5... Step: 13590... Loss: 1.4614... Val Loss: 1.4402
Epoch: 4/5... Step: 13600... Loss: 1.3627... Val Loss: 1.4394
Epoch: 4/5... Step: 13610... Loss: 1.4090... Val Loss: 1.4384
Epoch: 4/5... Step: 13620... Loss: 1.4751... Val Loss: 1.4461
Epoch: 4

Epoch: 5/5... Step: 14800... Loss: 1.3957... Val Loss: 1.4339
Epoch: 5/5... Step: 14810... Loss: 1.2985... Val Loss: 1.4253
Epoch: 5/5... Step: 14820... Loss: 1.3719... Val Loss: 1.4242
Epoch: 5/5... Step: 14830... Loss: 1.3787... Val Loss: 1.4248
Epoch: 5/5... Step: 14840... Loss: 1.4717... Val Loss: 1.4291
Epoch: 5/5... Step: 14850... Loss: 1.4609... Val Loss: 1.4256
Epoch: 5/5... Step: 14860... Loss: 1.3310... Val Loss: 1.4282
Epoch: 5/5... Step: 14870... Loss: 1.3042... Val Loss: 1.4285
Epoch: 5/5... Step: 14880... Loss: 1.5323... Val Loss: 1.4282
Epoch: 5/5... Step: 14890... Loss: 1.4388... Val Loss: 1.4273
Epoch: 5/5... Step: 14900... Loss: 1.5304... Val Loss: 1.4339
Epoch: 5/5... Step: 14910... Loss: 1.2735... Val Loss: 1.4304
Epoch: 5/5... Step: 14920... Loss: 1.3491... Val Loss: 1.4259
Epoch: 5/5... Step: 14930... Loss: 1.3591... Val Loss: 1.4272
Epoch: 5/5... Step: 14940... Loss: 1.3851... Val Loss: 1.4271
Epoch: 5/5... Step: 14950... Loss: 1.4311... Val Loss: 1.4221
Epoch: 5

Epoch: 5/5... Step: 16130... Loss: 1.3853... Val Loss: 1.4291
Epoch: 5/5... Step: 16140... Loss: 1.3577... Val Loss: 1.4252
Epoch: 5/5... Step: 16150... Loss: 1.3705... Val Loss: 1.4262
Epoch: 5/5... Step: 16160... Loss: 1.4952... Val Loss: 1.4223
Epoch: 5/5... Step: 16170... Loss: 1.4895... Val Loss: 1.4226
Epoch: 5/5... Step: 16180... Loss: 1.4329... Val Loss: 1.4249
Epoch: 5/5... Step: 16190... Loss: 1.4138... Val Loss: 1.4214
Epoch: 5/5... Step: 16200... Loss: 1.2791... Val Loss: 1.4241
Epoch: 5/5... Step: 16210... Loss: 1.3838... Val Loss: 1.4262
Epoch: 5/5... Step: 16220... Loss: 1.3340... Val Loss: 1.4250
Epoch: 5/5... Step: 16230... Loss: 1.5245... Val Loss: 1.4234
Epoch: 5/5... Step: 16240... Loss: 1.4414... Val Loss: 1.4228
Epoch: 5/5... Step: 16250... Loss: 1.5652... Val Loss: 1.4227
Epoch: 5/5... Step: 16260... Loss: 1.3384... Val Loss: 1.4208
Epoch: 5/5... Step: 16270... Loss: 1.4889... Val Loss: 1.4239
Epoch: 5/5... Step: 16280... Loss: 1.5092... Val Loss: 1.4238
Epoch: 5

Epoch: 5/5... Step: 17460... Loss: 1.3334... Val Loss: 1.4118
Epoch: 5/5... Step: 17470... Loss: 1.3226... Val Loss: 1.4077
Epoch: 5/5... Step: 17480... Loss: 1.3859... Val Loss: 1.4059
Epoch: 5/5... Step: 17490... Loss: 1.3850... Val Loss: 1.4126
Epoch: 5/5... Step: 17500... Loss: 1.3192... Val Loss: 1.4102
Epoch: 5/5... Step: 17510... Loss: 1.4377... Val Loss: 1.4074
Epoch: 5/5... Step: 17520... Loss: 1.2220... Val Loss: 1.4124
Epoch: 5/5... Step: 17530... Loss: 1.3419... Val Loss: 1.4140
Epoch: 5/5... Step: 17540... Loss: 1.3050... Val Loss: 1.4145
Epoch: 5/5... Step: 17550... Loss: 1.2972... Val Loss: 1.4160
Epoch: 5/5... Step: 17560... Loss: 1.2856... Val Loss: 1.4160
Epoch: 5/5... Step: 17570... Loss: 1.4932... Val Loss: 1.4138
Epoch: 5/5... Step: 17580... Loss: 1.3331... Val Loss: 1.4123
Epoch: 5/5... Step: 17590... Loss: 1.3886... Val Loss: 1.4152
Epoch: 5/5... Step: 17600... Loss: 1.3613... Val Loss: 1.4127
Epoch: 5/5... Step: 17610... Loss: 1.3666... Val Loss: 1.4134
Epoch: 5

## Getting the best model

To set your hyperparameters to get the best performance, you'll want to watch the training and validation losses. If your training loss is much lower than the validation loss, you're overfitting. Increase regularization (more dropout) or use a smaller network. If the training and validation losses are close, you're underfitting so you can increase the size of the network.

## Hyperparameters

Here are the hyperparameters for the network.

In defining the model:
* `n_hidden` - The number of units in the hidden layers.
* `n_layers` - Number of hidden LSTM layers to use.

We assume that dropout probability and learning rate will be kept at the default, in this example.

And in training:
* `batch_size` - Number of sequences running through the network in one pass.
* `seq_length` - Number of characters in the sequence the network is trained on. Larger is better typically, the network will learn more long range dependencies. But it takes longer to train. 100 is typically a good number here.
* `lr` - Learning rate for training

Here's some good advice from Andrej Karpathy on training the network. I'm going to copy it in here for your benefit, but also link to [where it originally came from](https://github.com/karpathy/char-rnn#tips-and-tricks).

> ## Tips and Tricks

>### Monitoring Validation Loss vs. Training Loss
>If you're somewhat new to Machine Learning or Neural Networks it can take a bit of expertise to get good models. The most important quantity to keep track of is the difference between your training loss (printed during training) and the validation loss (printed once in a while when the RNN is run on the validation data (by default every 1000 iterations)). In particular:

> - If your training loss is much lower than validation loss then this means the network might be **overfitting**. Solutions to this are to decrease your network size, or to increase dropout. For example you could try dropout of 0.5 and so on.
> - If your training/validation loss are about equal then your model is **underfitting**. Increase the size of your model (either number of layers or the raw number of neurons per layer)

> ### Approximate number of parameters

> The two most important parameters that control the model are `n_hidden` and `n_layers`. I would advise that you always use `n_layers` of either 2/3. The `n_hidden` can be adjusted based on how much data you have. The two important quantities to keep track of here are:

> - The number of parameters in your model. This is printed when you start training.
> - The size of your dataset. 1MB file is approximately 1 million characters.

>These two should be about the same order of magnitude. It's a little tricky to tell. Here are some examples:

> - I have a 100MB dataset and I'm using the default parameter settings (which currently print 150K parameters). My data size is significantly larger (100 mil >> 0.15 mil), so I expect to heavily underfit. I am thinking I can comfortably afford to make `n_hidden` larger.
> - I have a 10MB dataset and running a 10 million parameter model. I'm slightly nervous and I'm carefully monitoring my validation loss. If it's larger than my training loss then I may want to try to increase dropout a bit and see if that helps the validation loss.

> ### Best models strategy

>The winning strategy to obtaining very good models (if you have the compute time) is to always err on making the network larger (as large as you're willing to wait for it to compute) and then try different dropout values (between 0,1). Whatever model has the best validation performance (the loss, written in the checkpoint filename, low is good) is the one you should use in the end.

>It is very common in deep learning to run many different models with many different hyperparameter settings, and in the end take whatever checkpoint gave the best validation performance.

>By the way, the size of your training and validation splits are also parameters. Make sure you have a decent amount of data in your validation set or otherwise the validation performance will be noisy and not very informative.

## Checkpoint

After training, we'll save the model so we can load it again later if we need too. Here I'm saving the parameters needed to create the same architecture, the hidden layer hyperparameters and the text characters.

In [22]:
# change the name, for saving multiple files
model_name = 'rnn_5_epoch.net'

checkpoint = {'n_hidden': net.n_hidden,
              'n_layers': net.n_layers,
              'state_dict': net.state_dict(),
              'tokens': net.chars}

with open(model_name, 'wb') as f:
    torch.save(checkpoint, f)

---
## Making Predictions

Now that the model is trained, we'll want to sample from it and make predictions about next characters! To sample, we pass in a character and have the network predict the next character. Then we take that character, pass it back in, and get another predicted character. Just keep doing this and you'll generate a bunch of text!

### A note on the `predict`  function

The output of our RNN is from a fully-connected layer and it outputs a **distribution of next-character scores**.

> To actually get the next character, we apply a softmax function, which gives us a *probability* distribution that we can then sample to predict the next character.

### Top K sampling

Our predictions come from a categorical probability distribution over all the possible characters. We can make the sample text and make it more reasonable to handle (with less variables) by only considering some $K$ most probable characters. This will prevent the network from giving us completely absurd characters while allowing it to introduce some noise and randomness into the sampled text. Read more about [topk, here](https://pytorch.org/docs/stable/torch.html#torch.topk).


In [23]:
def predict(net, char, h=None, top_k=None):
        ''' Given a character, predict the next character.
            Returns the predicted character and the hidden state.
        '''
        
        # tensor inputs
        x = np.array([[net.char2int[char]]])
        x = one_hot_encode(x, len(net.chars))
        inputs = torch.from_numpy(x)
        
        if(train_on_gpu):
            inputs = inputs.cuda()
        
        # detach hidden state from history
        h = tuple([each.data for each in h])
        # get the output of the model
        out, h = net(inputs, h)

        # get the character probabilities
        p = F.softmax(out, dim=1).data
        if(train_on_gpu):
            p = p.cpu() # move to cpu
        
        # get top characters
        if top_k is None:
            top_ch = np.arange(len(net.chars))
        else:
            p, top_ch = p.topk(top_k)
            top_ch = top_ch.numpy().squeeze()
        
        # select the likely next character with some element of randomness
        p = p.numpy().squeeze()
        char = np.random.choice(top_ch, p=p/p.sum())
        
        # return the encoded value of the predicted char and the hidden state
        return net.int2char[char], h

### Priming and generating text 

Typically you'll want to prime the network so you can build up a hidden state. Otherwise the network will start out generating characters at random. In general the first bunch of characters will be a little rough since it hasn't built up a long history of characters to predict from.

In [24]:
def sample(net, size, prime='The', top_k=None):
        
    if(train_on_gpu):
        net.cuda()
    else:
        net.cpu()
    
    net.eval() # eval mode
    
    # First off, run through the prime characters
    chars = [ch for ch in prime]
    h = net.init_hidden(1)
    for ch in prime:
        char, h = predict(net, ch, h, top_k=top_k)

    chars.append(char)
    
    # Now pass in the previous character and get a new one
    for ii in range(size):
        char, h = predict(net, chars[-1], h, top_k=top_k)
        chars.append(char)

    return ''.join(chars)

In [25]:
print(sample(net, 1000, prime='Anna', top_k=5))

Anna,"
she said.

"Yes, that if there is something and a signiful at the pain whether I shall be answer to be, this house a mind and sat a lust, and to mo be so mental, that you can't see that she went up to there as the court and many hand one that is to say that his from the whele, but he was so standing, what will the line through she had been, and this house that she was a mind," she
thought, she started to the peite she, and so stayed and sound and himed of her son and the carriage, and she had say story a mistoraberal shame
the soul of the
father, and that he was a figure to her.

"I do not all an exastering of the
crowd with a little sate to more."

"You made the
mide with you."

"No, as the princess were at hare in the soun at humor? Worn anything the secants of this wife,
and how altake more in him it as that
I don't do nothing but he want to say." She crossed him, and had been that her that would have been
a secret, what start and take his face, he was a face of the port of a

## Loading a checkpoint

In [27]:
# Here we have loaded in a model that trained over 20 epochs `rnn_20_epoch.net`
with open('rnn_5_epoch.net', 'rb') as f:
    checkpoint = torch.load(f)
    
loaded = CharRNN(checkpoint['tokens'], n_hidden=checkpoint['n_hidden'], n_layers=checkpoint['n_layers'])
loaded.load_state_dict(checkpoint['state_dict'])

<All keys matched successfully>

In [28]:
# Sample using a loaded model
print(sample(loaded, 2000, top_k=5, prime="And Levin said"))

And Levin said, and always
came to a shame and and her, and that to him her and there with the conscious, would be already
so one with seems was to tell at the princess
had.

The candor and at the hand of a formations, at once
her husband that they could not care in the call of the men which she had some now at the follow, but talking of world him of the same to her so into as a same
to his feeling who was nothered; but it was not breaking
that they could
not heard
herself in his white
short, because that it was sharply
and thought of supposing, who had not all a same words,
ald his being to them out of the came out of the feeling
of his bare that he could not say that she had not an old mustracter, and another has been at their family. But that she could not be a side of soritian on his while about the side, at itself to him.

"That it was no was the peasants. There's not
been to
be death, I we trouble himself of a letter, and what is it was it," answered Karenin. "We shan to tell," a